In [326]:
import re
import pandas as pd
import subprocess
import numpy as np


In [327]:
class ParameterOptmizer:
    def __init__(self):
        # define the experimental data
        self.exp_list = ['H2', 'CH4', 'C2H6', 'C2H4', 'C2H2', 'C3H8', 'C3H6', 'C4H10', 'C5H12', 
                        'C', 'CH', 'CH2', 'CH3', 'C2H3', 'C2H5', 'C3H7', 'H',
                        'CH3^+', 'CH4^+', 'CH5^+', 'C2H2^+', 'C2H4^+', 'C2H5^+', 'C2H6^+', 'C3H6^+', 'C3H8^+']
        self.exp_values = [5.319082, 90.80403, 2.428802, 0.197735, 0.171795, 0.717088, 0.046734, 
                          0.114829, 0.119573, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        # define the file path
        self.kinet_path = 'kinet.inp'
        self.exe_path = 'run2.exe'

        # dictionary to save the current parameters
        self.current_parameters = {}
        self.load_current_parameters(init=True)
    
    def load_current_parameters(self, init=True):
        # 시작인 경우 kinet_ori.inp에서 parameter를 불러옴
        if init:
            with open('kinet_ori.inp', 'r') as f:
                content = f.readlines()
        else:
            # kinet.inp 파일에서 현재 파라미터 값들을 읽어옴
            with open(self.kinet_path, 'r') as f:
                content = f.readlines()

        for line in content:
            if "$ double precision, parameter :: f" in line:
                # f parameter와 값 추출
                pattern = r'f(\d+)\s*=\s*([\d\.\+\-d]+)'
                match = re.search(pattern, line)
                if match:
                    param_name = 'f' + match.group(1)
                    value = float(match.group(2).replace('d','e'))
                    self.current_parameters[param_name] = value

    def modify_parameter(self, param_name: str, new_value: float) -> None:
        # modify the parameter value in the kinet.inp file
        with open (self.kinet_path, 'r') as f:
            content = f.read()
        
        # modify the parameter
        pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
        new_value_str = f'{new_value:.4e}'.replace('e', 'd')
        content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)

        with open(self.kinet_path, 'w') as f:
            f.write(content)
    
    def run_preprocessor(self):
        process = subprocess.Popen(  # 외부 프로세스 실행을 위한 Popen 객체 생성
        './preprocessor.exe',    # 실행할 프로그램 경로 지정
        stdin=subprocess.PIPE,   # 표준 입력 파이프 설정 - 프로세스에 입력을 전달하기 위함
        stdout=subprocess.PIPE,  # 표준 출력 파이프 설정 - 프로세스의 출력을 받기 위함
        stderr=subprocess.PIPE,  # 표준 에러 파이프 설정 - 프로세스의 에러를 받기 위함
        universal_newlines=False) # 텍스트 모드로 파이프 처리 - 문자열을 자동으로 인코딩/디코딩

        process.stdin.write(f'{self.kinet_path}\n'.encode())  # 입력 파일 경로를 프로세스에 전달
        process.stdin.flush()  # 버퍼를 비워서 데이터가 즉시 전송되도록 함

        process.stdin.write('.\n'.encode())  # 현재 디렉토리 표시를 프로세스에 전달
        process.stdin.flush()  # 버퍼를 비워서 데이터가 즉시 전송되도록 함

        output, error = process.communicate()  # 프로세스 실행 완료를 기다리고 출력과 에러를 받음
        output_str = output.decode('utf-8', errors='ignore') if output else ''
        error_str = error.decode('utf-8', errors='ignore') if error else ''
    
        print('check the run of preprocessor')  # 전처리기 실행 확인 메시지 출력 
        return output_str, error_str
    
    def compile_zdp(self):
        compile_command = [  # 컴파일 명령어 리스트 생성
        'gfortran', '-o', self.exe_path, 'dvode_f90_m.F90', 'zdplaskin_m.F90', 'run2.F90', 'bolsig_x86_64_g.dll'
        ]
        result = subprocess.run(compile_command, capture_output=True, text=True)  # 컴파일 명령 실행
    
        if result.returncode != 0:  # 컴파일 결과 확인
            raise Exception(f"{self.exe_path} 컴파일 실패")  # 컴파일 실패시 예외 발생
        print('check the compile_zdp')  # 컴파일 완료 메시지 출력

    def run_simulation(self):
        try:
            process = subprocess.Popen(  # 실행 파일 실행
                self.exe_path,    
                stdout=subprocess.PIPE,  
                stderr=subprocess.STDOUT,  
                stdin=subprocess.PIPE,   
                universal_newlines=True,
                bufsize=1,              
            )
            
            while True:
                output = process.stdout.readline()
                
                if not output:
                    break
                    
                print(f'\r{output.strip()}                                                                              ',end='',flush=True)
                
                if "PRESS ENTER TO EXIT" in output:
                    print()
                    process.kill()
                    break

                if "WARNING: BOLSIG+ convergence failed" in output:
                    process.stdin.write('\n')
                    process.stdin.flush()

        except:
            pass
        return process
    
    def err_calculation(self):
        # read the result file
        species = []
        with open('qt_species_list.txt', 'r') as f:
            for line in f:
                comp = line[2:]
                species.append(comp.strip())
        
        df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
        
        # calculate the concentration
        H2 = (df_sp['H2'])
        CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
        C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'] + df_sp['C2H2(V13)'])
        C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])
        C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])
        C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
        C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
        C4H10 = (df_sp['C4H9H'])
        C5H12 = (df_sp['C5H12'])
        C = (df_sp['C'])
        CH = (df_sp['CH'])
        CH2 = (df_sp['CH2'])
        CH3 = (df_sp['CH3'])
        C2H3 = (df_sp['C2H3'])
        C2H5 = (df_sp['C2H5'])
        C3H7 = (df_sp['C3H7'])
        H = (df_sp['H'])
        CH3_plus = (df_sp['CH3^+'])
        CH4_plus = df_sp['CH4^+']
        CH5_plus = df_sp['CH5^+']
        C2H2_plus = df_sp['C2H2^+']
        C2H4_plus = df_sp['C2H4^+']
        C2H5_plus = df_sp['C2H5^+']
        C2H6_plus = df_sp['C2H6^+']
        C3H6_plus = df_sp['C3H6^+']
        C3H8_plus = df_sp['C3H8^+']

        all_sp = df_sp.sum(axis=1) - df_sp['E']

        t = abs(df_sp['Time [s]']-16.96).argmin()

        sim_H2 = float(format(H2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH4 = float(format(CH4.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H2 = float(format(C2H2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H4 = float(format(C2H4.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H6 = float(format(C2H6.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C3H6 = float(format(C3H6.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C3H8 = float(format(C3H8.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C4H10 = float(format(C4H10.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C5H12 = float(format(C5H12.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C = float(format(C.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH = float(format(CH.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH2 = float(format(CH2.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH3 = float(format(CH3.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H3 = float(format(C2H3.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H5 = float(format(C2H5.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C3H7 = float(format(C3H7.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_H = float(format(H.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH3_plus = float(format(CH3_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH4_plus = float(format(CH4_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_CH5_plus = float(format(CH5_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H2_plus = float(format(C2H2_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H4_plus = float(format(C2H4_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H5_plus = float(format(C2H5_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C2H6_plus = float(format(C2H6_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C3H6_plus = float(format(C3H6_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))
        sim_C3H8_plus = float(format(C3H8_plus.iloc[t]/all_sp.iloc[t]*100, '.6f'))

        sim = []
        sim.append(sim_H2)
        sim.append(sim_CH4)
        sim.append(sim_C2H6)
        sim.append(sim_C2H4)
        sim.append(sim_C2H2)
        sim.append(sim_C3H8)
        sim.append(sim_C3H6)
        sim.append(sim_C4H10)
        sim.append(sim_C5H12)
        sim.append(sim_C)
        sim.append(sim_CH)
        sim.append(sim_CH2)
        sim.append(sim_CH3)
        sim.append(sim_C2H3)
        sim.append(sim_C2H5)
        sim.append(sim_C3H7)
        sim.append(sim_H)
        sim.append(sim_CH3_plus)
        sim.append(sim_CH4_plus)
        sim.append(sim_CH5_plus)
        sim.append(sim_C2H2_plus)
        sim.append(sim_C2H4_plus)
        sim.append(sim_C2H5_plus)
        sim.append(sim_C2H6_plus)
        sim.append(sim_C3H6_plus)
        sim.append(sim_C3H8_plus)        
        
        err = 0
        for i in range(len(self.exp_values)):
            err += ((self.exp_values[i] - sim[i]))**2
        err -= (self.exp_values[8] - sim[8])**2

        return err, df_sp['Time [s]'].iloc[-1]  

    def LH_sampling(self, n_samples=20):
        # load a current parameter set
        paraset = self.current_parameters

        # define the bounds
        bounds = np.array([[-0.1,0.1]] * len(paraset))

        # sampling
        samples = np.random.uniform(bounds[:,0], bounds[:,1], (n_samples, len(paraset)))

        return samples
        
        
    

In [328]:
if __name__ == "__main__":
    optimizer = ParameterOptmizer()
    max_error = 100
    # iniitialization
    optimizer.load_current_parameters(init=True)
    param = np.array(list(optimizer.current_parameters.values()))
    for i in range(len(param)):
        optimizer.modify_parameter(f'f{i}', param[i])
    optimizer.run_preprocessor()
    optimizer.compile_zdp()
    optimizer.run_simulation()

    res_init = optimizer.current_parameters.copy()
    res_init['err'] = optimizer.err_calculation()[0]
    res_init['index'] = 'initialization'

    df_set = pd.DataFrame([res_init])
    df_set.to_csv('db_set.csv', index=False)
    print(f'State: initialization Done')

    # LH sample pre-train data
    param_LH = 10**optimizer.LH_sampling()
    param_init = np.array(list(optimizer.current_parameters.values()))
    for j in range(len(optimizer.LH_sampling())):
        param = param_LH[j] * param_init
        for i in range(len(param)):
            optimizer.modify_parameter(f'f{i}', param[i])
        optimizer.load_current_parameters(init=False)
        optimizer.run_preprocessor()
        optimizer.compile_zdp()
        optimizer.run_simulation()

        res = optimizer.current_parameters.copy()
        total_time = optimizer.err_calculation()[1]
        
        if total_time > 16.96:
            res['err'] = optimizer.err_calculation()[0]  
        else:
            res['err'] = max_error
        res['index'] = f'LH pre-train {j}'
        
        print(f'State: LH pre-train, iteration: {j}, error = {res['err']}')
        df_set = pd.concat([df_set, pd.DataFrame([res])], ignore_index=True)
        df_set.to_csv('db_set.csv', index=False)




check the run of preprocessor
check the compile_zdp
PRESS ENTER TO EXIT ...At line 88 of file run2.F90 (unit = 5, file = 'fort.5')                                                                                         
State: initialization Done
check the run of preprocessor
check the compile_zdp
PRESS ENTER TO EXIT ...At line 88 of file run2.F90 (unit = 5, file = 'fort.5')                                                                                         
State: LH pre-train, iteration: 0, error = 48.346193030734035
check the run of preprocessor
check the compile_zdp
PRESS ENTER TO EXIT ...At line 88 of file run2.F90 (unit = 5, file = 'fort.5')                                                                                         
State: LH pre-train, iteration: 1, error = 5.238241732379003
check the run of preprocessor
check the compile_zdp
PRESS ENTER TO EXIT ...At line 88 of file run2.F90 (unit = 5, file = 'fort.5')                                                            

In [329]:
res_init

{'f0': 1.0,
 'f1': 1.4e-10,
 'f2': 10000000.0,
 'f3': 1.0,
 'f4': 1.0,
 'f5': 100000000.0,
 'f6': 1.0,
 'f7': 1.0,
 'f8': 10.0,
 'f9': 1.0,
 'f10': 1.0,
 'f11': 0.01,
 'f12': 1.0,
 'f13': 1.0,
 'f14': 1.0,
 'f15': 1.0,
 'f16': 100000000.0,
 'f17': 1.0,
 'f18': 100000000.0,
 'f19': 1.0,
 'f20': 1.0,
 'f21': 1.0,
 'f22': 1.0,
 'f23': 1.0,
 'f24': 1.0,
 'f25': 1.0,
 'f26': 1.0,
 'f27': 1.0,
 'f28': 1.0,
 'f29': 1.0,
 'f30': 1.0,
 'f31': 1.0,
 'f32': 1.0,
 'f33': 1.0,
 'f34': 10.0,
 'f35': 10000000000.0,
 'f36': 1.0,
 'err': 6.344282229282006,
 'index': 'initialization'}

In [330]:

with open ('kinet.inp', 'r') as f:
    content = f.read()

param_name = 'f2'
new_value = 1e1
print(new_value)
# modify the parameter
pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
print(pattern)
new_value_str = f'{new_value:.4e}'.replace('e-0','e-').replace('e+00','e0').replace('e+0','e+').replace('e+','e').replace('e','d')
print(new_value_str)
content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)

content


10.0
(parameter :: f2 = )([\d\.\+\-d]+)
1.0000d1


'ELEMENTS\ne C H\nEND\n\nSPECIES\ne C CH CH2 CH3 CH3^+ CH4 CH4(v13) CH4(v24) CH4^+ CH5^+ C2H2 C2H2(v13) C2H2(v2) C2H2(v5) C2H2^+ C2H3 C2H4 C2H4(v1) C2H4(v2) C2H4^+ C2H5 C2H5^+ \nC2H6 C2H6(v13) C2H6(v24) C2H6^+ C3H6 C3H6(v) C3H6^+ C3H7 C3H8 C3H8(v1) C3H8(v2) C3H8^+ C4H9H C5H12 H H2\nEND\n\nBOLSIG\nCH CH3 CH3^+ CH4 CH4(v13) CH4(v24) CH4^+ C2H2 C2H2(v13) C2H2(v2) C2H2(v5) C2H2^+ C2H4 C2H4(v1) C2H4(v2) C2H4^+ C2H5^+ \nC2H6 C2H6(v13) C2H6(v24) C2H6^+ C3H6 C3H6(v) C3H6^+ C3H8 C3H8(v1) C3H8(v2) C3H8^+\nEND\n\nREACTIONS\n# Gas Constant\n$ double precision, parameter :: R = 8.314d-3\n\n# Compensation Parameters\n$ double precision, parameter :: f0 = 1.0319d+00\n$ double precision, parameter :: f1 = 1.5263d-10\n$ double precision, parameter :: f2 = 1.0000d1\n$ double precision, parameter :: f3 = 1.0894d+00\n$ double precision, parameter :: f4 = 1.1011d+00\n$ double precision, parameter :: f5 = 1.1594d+08\n$ double precision, parameter :: f6 = 8.2379d-01\n$ double precision, parameter :: f7 = 1.1